# Grafico interattivo: `sample_size` vs `f0_hat_mean`

Questo notebook crea un grafico interattivo con **hover** sui punti e sulle linee interpolate.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.graph_objects as go

candidate_paths = [Path('results.csv'), Path('../results.csv')]
csv_path = next((p for p in candidate_paths if p.exists()), None)
if csv_path is None:
    raise FileNotFoundError('results.csv non trovato (cercato in cwd e parent).')

df = pd.read_csv(csv_path)
required_cols = {'algorithm', 'sample_size', 'f0_hat_mean'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f'Colonne mancanti nel CSV: {missing}')

df[['sample_size', 'f0_hat_mean']] = df[['sample_size', 'f0_hat_mean']].apply(pd.to_numeric, errors='coerce')
plot_df = (df.dropna(subset=['algorithm', 'sample_size', 'f0_hat_mean'])
             .groupby(['algorithm', 'sample_size'], as_index=False)['f0_hat_mean']
             .mean()
             .sort_values(['algorithm', 'sample_size']))

print(f'CSV: {csv_path.resolve()}')
print(f'Righe originali: {len(df)} | Righe aggregate: {len(plot_df)}')
plot_df.head()

In [ ]:
fig = go.Figure()

for algo, g in plot_df.groupby('algorithm'):
    g = g.sort_values('sample_size')
    x = g['sample_size'].to_numpy(dtype=float)
    y = g['f0_hat_mean'].to_numpy(dtype=float)

    # Punti originali con hover dettagliato
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name=f'{algo} (punti)',
        marker=dict(size=9),
        customdata=np.array([[algo]] * len(x)),
        hovertemplate='algoritmo: %{customdata[0]}<br>sample_size: %{x}<br>f0_hat_mean: %{y:.6f}<extra></extra>'
    ))

    # Interpolazione lineare tra i punti
    if len(x) >= 2:
        x_dense = np.linspace(x.min(), x.max(), 400)
        y_dense = np.interp(x_dense, x, y)
        fig.add_trace(go.Scatter(
            x=x_dense,
            y=y_dense,
            mode='lines',
            name=f'{algo} (interp)',
            line=dict(width=2),
            customdata=np.array([[algo]] * len(x_dense)),
            hovertemplate='algoritmo: %{customdata[0]} (interp)<br>sample_size: %{x:.2f}<br>f0_hat_mean: %{y:.6f}<extra></extra>'
        ))

fig.update_layout(
    title='f0_hat_mean vs sample_size per algoritmo (interattivo)',
    xaxis_title='sample_size',
    yaxis_title='f0_hat_mean',
    hovermode='closest',
    template='plotly_white',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0),
)


try:
    fig.show()
except ValueError as e:
    if 'nbformat' in str(e):
        import plotly.io as pio
        pio.renderers.default = "browser"
        fig.show()
    else:
        raise


Se serve salvare in HTML interattivo:

```python
fig.write_html('f0_hat_vs_sample_size_interactive.html', include_plotlyjs='cdn')
```